Merge `GPT-3 log` data between 2 sqlite databases

In [1]:
import sqlite3
import pandas as pd

DELIMITOR = ","

class DBConn(object):
    def __init__(self, db_file):
        self.conn = sqlite3.connect(db_file)
    def __enter__(self):
        return self.conn
    def __exit__(self, type, value, traceback):
        self.conn.close()

def get_data(db_file, table_name):
    with DBConn(db_file) as _conn:
        sql_stmt = f'''
        SELECT uuid||'{DELIMITOR}'||ts as uuid_ts from {table_name};
        '''
        return pd.read_sql(sql_stmt, _conn)

def list2sql_str(l):
    """convert a list into SQL in string
    """
    return str(l).replace("[", "(").replace("]", ")")

def merge_logs(src_db, tgt_db="gpt3sql.sqlite", table_name="t_gpt3_log"):
    
    df_src = get_data(src_db, table_name)
    df_tgt = get_data(tgt_db, table_name)

    uuid_ts_src = df_src["uuid_ts"].to_list()
    uuid_ts_tgt = df_tgt["uuid_ts"].to_list()

    uuid_update = [i.split(DELIMITOR)[0] for i in (set(uuid_ts_src) - set(uuid_ts_tgt))]

    if uuid_update:

        # fetch new/updated row from src
        with DBConn(src_db) as _conn:
            sql_stmt = f'''
            SELECT * from {table_name} where uuid in {list2sql_str(uuid_update)} ;
            '''
            df_src = pd.read_sql(sql_stmt, _conn)

        with DBConn(tgt_db) as _conn:
            # remove old rows in tgt
            delete_sql = f"""
                delete from {table_name} where uuid in {list2sql_str(uuid_update)} ;
            """
            cur = _conn.cursor()
            cur.executescript(delete_sql)
            _conn.commit()

            # append to tgt
            df_src.to_sql(table_name, _conn, if_exists='append', index=False)
            _conn.commit()  

    return uuid_update

In [2]:
src_db = "gpt3sql-20221123.sqlite"
uuid_update = merge_logs(src_db)
print(uuid_update)

['aab944b6-1b07-4738-92b8-230564ad49c0', '78d7f638-0b42-415a-a5b8-b093135a6034', '600d4d62-b165-43bd-85e0-dc962f50c3cd', 'b2e174f1-82a6-470b-ad4e-1967e142687f', 'fc2fed9c-af94-4098-b011-c5995fd5fb41', '90461d42-44bb-4028-9eef-123b34c84e61', '757a57d4-e7b0-4159-a6a0-abd961511498']
